# Build a Question/Answering system over SQL data

Enabling a LLM system to query structured data (e.g., database) is very different from unstructured text data. For **unstructured** text data, we generate text that can be searched against a vector database. For **structured** data, we need the LLM to write and execute queries in SQL. 

In this guide we'll go over the basic ways to create a Q&A system over tabular data in databases. We will cover implementations using both **chains** and an **agent**. These systems will allow us to ask a question about the data in a database and get back a natural language answer. The main difference between chains and an agent is that our agent can query the database in a loop as many times as it needs to answer the question.


> ## ⚠️ Security note ⚠️
>
> Building Q&A systems of SQL databases requires executing model-generated SQL queries. There are inherent risks in doing this. Make sure that your database connection permissions are always scoped as narrowly as possible for your agent's needs. This will reduce though **not** eliminate the risks of building a model-driven system. For more on general security best practices, see [LangChain Security Docs](https://python.langchain.com/v0.2/docs/security).

The code in this notebook is adapted from the LangChain tutorial: [Build a Question/Answering system over SQL data](https://python.langchain.com/v0.2/docs/tutorials/sql_qa).


## Architecture

At a high-level, the steps of these systems are:

1. **Convert question to SQL query**: Model converts user's query to a SQL query.
2. **Execute SQL query**: Execute the SQL query.
3. **Answer the question**: Model responds to user input using the database query results.


![Convert user's question to SQL](img/sql_usecase.png "Convert user's question to SQL")
**Figure 1**: Image from [LangChain tutorial](https://python.langchain.com/v0.2/docs/tutorials/sql_qa/#architecture)


## Setup

Before we begin, please make sure you have setup the `.env` file in the project 
directory as described in [`README.md`](README.md).

Next, we will load in the necessary environment variables (e.g., API keys) for this notebook:

In [4]:
import os
from dotenv import load_dotenv

_ = load_dotenv()

assert os.environ.get("GOOGLE_API_KEY")

## Database

In this notebook we will use a SQLite connection with the [Chinook](https://github.com/lerocha/chinook-database) sample database. The database has already been created for you in the `db` directory.

![Chinook database ER diagram](img/chinook-database-er-diagram.png "Chinook database ER diagram")
**Figure 2**: Image from [`chinook-database` GitHub](https://github.com/lerocha/chinook-database)

We can interface with the database using the SQLAlchemy-driven [`SQLDatabase`](https://python.langchain.com/v0.2/api_reference/community/utilities/langchain_community.utilities.sql_database.SQLDatabase.html) class:

In [2]:
from langchain_community.utilities import SQLDatabase

# Open database in read-only (`ro`) mode.
database = SQLDatabase.from_uri(
    "sqlite:///db/Chinook.db?mode=ro"
)

print(f"SQL Dialect: {database.dialect}")
print(
    f"Tables: {database.get_usable_table_names()}"
)

SQL Dialect: sqlite
Tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [3]:
database.run("SELECT * FROM Artist LIMIT 10;")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

Great! We've got a SQL database that we can query. Now let's try hooking it up to an LLM.


## Chain

Chains (i.e., compositions of LangChain [Runnables](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel) support applications whose steps are predictable. We can create a simple chain that takes a question and does the following:
1. Convert the question into a SQL query.
2. Execute the SQL query.
3. Use the results to answer the original question.

There are scenarios not supported by this arrangement. For example, this system will execute a SQL query for any user input — even `"hello"`. Importantly, as we'll see below, some questions require more than one query to answer. We will address these scenarios in the [Agent](#Agent) section.

### Convert question to SQL query

Let's initialize a LLM that we will use in the chain. We will be using Google's [**Gemini 1.5 Flash**](https://ai.google.dev/gemini-api/docs/models/gemini#gemini-1.5-flash), since it offers a generous free tier for us to play around with.

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", 
    temperature=0
)

The first step in a SQL chain or agent is to take the user input and convert it to a SQL query. LangChain comes with a built-in chain for this: [`create_sql_query_chain`](https://python.langchain.com/v0.2/api_reference/langchain/chains/langchain.chains.sql_database.query.create_sql_query_chain.html).

In [23]:
from langchain.chains import create_sql_query_chain

write_query = create_sql_query_chain(
    llm=llm, 
    db=database
)

response = write_query.invoke(
    input={
        "question": "How many employees are there?"
    }
)

response

'```sqlite\nSELECT COUNT(*) FROM Employee\n```'

If we try executing the generated SQL directly, we will encounter a syntax error:

In [24]:
try:
    print(
        database.run(response)
    )
except Exception as error:
    print(error)

(sqlite3.OperationalError) near "```sqlite
SELECT COUNT(*) FROM Employee
```": syntax error
[SQL: ```sqlite
SELECT COUNT(*) FROM Employee
```]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


This is because the LLM generates the SQL with `sqlite` tags. Instead, we need to extract a valid SQL from the response. To do this, we will make the LLM return structured output.

> **See also**: [How to return structured data from a model](https://python.langchain.com/v0.2/docs/how_to/structured_output/)

In [25]:
from langchain_core.pydantic_v1 import (
    BaseModel, 
    Field
)

class OutputFormat(BaseModel):
    """
    The expected output format.    
    """
    question: str = Field(
        description=(
            "The user's question."
        )
    )
    query: str = Field(
        description=(
            "A valid SQL query that can be "
            "executed against a SQLite database "
            "to answer the user's question."            
        )
    )

In [32]:
# Chain to extract a valid SQL from the 
# generated response
parse_sql = (
    llm.with_structured_output(
        schema=OutputFormat
    )
    | (lambda output: output.query)
)

chain = (
    write_query 
    | parse_sql
)

response = chain.invoke(
    input={
        "question": "How many employees are there?"
    }
)

response

'SELECT COUNT(*) FROM Employee'

Now, we can successfully execute the SQL query:

In [31]:
try:
    print(
        database.run(response)
    )
except Exception as error:
    print(error)

[(8,)]


We can look at the [LangSmith trace](https://smith.langchain.com/public/c8fa52ea-be46-4829-bde2-52894970b830/r) to get a better understanding of what this chain is doing. We can also inspect the chain directly for its prompts. Looking at the prompt (below), we can see that it is:

* Dialect-specific. In this case it references SQLite explicitly.
* Has definitions for all the available tables.
* Has three examples rows for each table.

This technique is inspired by papers like [this](https://arxiv.org/pdf/2204.00498.pdf), which suggest showing examples rows and being explicit about tables improves performance. We can also inspect the full prompt like so:

In [33]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### Query Validation

The most error-prone part of any SQL chain or agent is writing valid and safe SQL queries. We can make use of LangChain's built-in [`QuerySQLCheckerTool`](https://python.langchain.com/v0.2/api_reference/community/tools/langchain_community.tools.sql_database.tool.QuerySQLCheckerTool.html) that uses an LLM to check the SQL query for common mistakes.

> **See also**: [How to do query validation](https://python.langchain.com/v0.2/docs/how_to/sql_query_checking/)

In [44]:
from langchain_community.tools import QuerySQLCheckerTool

check_query = QuerySQLCheckerTool(
    db=database, 
    llm=llm
)

Let's check out the prompt that is used for checking the query:

In [45]:
print(check_query.template)


{query}
Double check the {dialect} query above for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

Output the final SQL query only.

SQL Query: 


Try running it on an incorrect SQL query (❌`NOT IN NULL` instead of ✅`IS NOT NULL`):

In [49]:
check_query.invoke(
    input={
        "query": "SELECT * FROM Employee WHERE LastName NOT IN NULL;"
    }
)

'```sqlite\nSELECT * FROM Employee WHERE LastName IS NOT NULL;\n```'

We can see that the LLM corrected our query. Let's move on to executing the query.

### Execute SQL query

Now that we've generated a SQL query, we'll want to execute it. **This is the most dangerous part of creating a SQL chain.** Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to your chains before query execution (see below).

We can use the [`QuerySQLDatabaseTool`](https://python.langchain.com/v0.2/api_reference/community/tools/langchain_community.tools.sql_database.tool.QuerySQLDataBaseTool.html) to easily add query execution to our chain:

In [53]:
from langchain_community.tools.sql_database.tool import (
    QuerySQLDataBaseTool
)

execute_query = QuerySQLDataBaseTool(
    db=database
)

chain = (
    write_query 
    | check_query
    | parse_sql
    | execute_query
)

response = chain.invoke(
    input={
        "question": "How many employees are there?"
    }
)

response

'[(8,)]'

### Answer the question

Now that we've got a way to automatically generate and execute queries, we just need to combine the original question and SQL query result to generate a final answer. We can do this by passing question and result to the LLM once more:

In [58]:
from langchain_core.prompts import PromptTemplate

answer_prompt = PromptTemplate.from_template(
    "Given the following user question, "
    "corresponding SQL query, and SQL result, "
    "answer the user question.\n\n" 
    "Question: {question}\n"
    "SQL Query: {query}\n"
    "SQL Result: {result}\n"
    "Answer: "
)

print(answer_prompt.template)

Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: 


In [59]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


chain = (
    RunnablePassthrough.assign(
        query=(
            write_query 
            | check_query 
            | parse_sql
        )
    ).assign(
        result=(
            itemgetter("query") 
            | execute_query
        )
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke(
    input={
        "question": "How many employees are there?"
    }
)

response

'There are **8** employees. \n'

Let's review what is happening in the above LangChain Expression Language (LCEL). When this chain is invoked:

- After the first `RunnablePassthrough.assign`, we have a runnable with two elements:
    ```python
    {
        "question": "How many employees are there?",
        "query": write_query.invoke(question)
    }
    ```
where `write_query` will generate a SQL query to answer the user's `question`.

- After the second `RunnablePassthrough.assign`, we have add a third element `"result"` that contains `execute_query.invoke(query)`, where `query` was computed in the previous step.
    ```python
    {
        "question": "How many employees are there?",
        "query": write_query.invoke(question),
        "result": execute_query.invoke(query)
    }
    ```
    
- These three inputs are formatted into the prompt and passed into the LLM.
- The `StrOutputParser()` plucks out the string content of the output message.

Note that we are composing LLMs, tools, prompts, and other chains together, but because each implements the `Runnable` interface, their inputs and outputs can be tied together in a reasonable way.

> **See also**: [LangChain Expression Language Cheatsheet](https://python.langchain.com/v0.2/docs/how_to/lcel_cheatsheet/)

## Agent